In [2]:
import pandas as pd
import numpy as np
import pyodbc
from scipy.stats import shapiro
from scipy.stats import kstest
from datetime import datetime
from configparser import ConfigParser

In [3]:
pd.set_option("display.max_column",200)
pd.set_option("display.float_format",lambda x:'%.5f' %x)

In [4]:
data_ind_U = pd.read_csv(r"/data/peer_updated_Incremental.csv")

In [5]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [7]:
risk_lkp = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES]", conn)
auto_high_rules =  risk_lkp[(risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['RISK_TYPE'] == 'Auto High Rules ')]['RULE_NO'].to_list()
static_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 1) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Individual')]['RULE_NO'].to_list()
non_stat_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 0) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Individual')]['RULE_NO'].to_list()
# non_stat_rules = tuple(non_stat_rules)
stat_rules = tuple(static_rules)

In [8]:
et_risk_df_per = pd.read_sql_query("SELECT * FROM FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG A inner join FINCORE_DB.Fincore_Analytics.RISK_PERSON_INCREMENTAL B ON A.PERSON_MASTER_ID = B.PERSON_MASTER_ID", conn)

In [ ]:
if len(non_stat_rules) == 1:
    et_risk_df_per_rules = pd.read_sql_query("SELECT A.* FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES]  A with(nolock) inner join FINCORE_DB.Fincore_Analytics.RISK_PERSON_INCREMENTAL B with (nolock) ON A.PERSON_MASTER_ID = B.PERSON_MASTER_ID where RULE_NO in ('{}')".format(non_stat_rules[0]),conn)
    
elif(len(non_stat_rules)== 0):
    et_risk_df_per_rules = pd.read_sql_query("SELECT A.* FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES]  A with(nolock) inner join FINCORE_DB.Fincore_Analytics.RISK_PERSON_INCREMENTAL B with (nolock) ON A.PERSON_MASTER_ID = B.PERSON_MASTER_ID where RULE_NO in ('')",conn)
    
else:
    non_stat_rules = tuple(non_stat_rules)
    et_risk_df_per_rules = pd.read_sql_query("SELECT A.* FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES]  A with(nolock) inner join FINCORE_DB.Fincore_Analytics.RISK_PERSON_INCREMENTAL B with (nolock) ON A.PERSON_MASTER_ID = B.PERSON_MASTER_ID where RULE_NO in" + str (non_stat_rules) , conn)

# stat_rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] where RULE_NO in "+str(tuple(static_rules)) , conn)


In [12]:
auto_high_df = et_risk_df_per_rules[(et_risk_df_per_rules['RULE_NO'].isin(auto_high_rules)) & (et_risk_df_per_rules['RISK_SCORE_STATIC'] == 10)]
risk_per = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON]",conn)
auto_high_df = auto_high_df.groupby('PERSON_MASTER_ID')['RULE_NO'].unique().reset_index()
auto_high_df['RISK_RULE_LIST'] = auto_high_df['RULE_NO'].apply(lambda x:"_".join(x))
auto_high_df['RISK_SCORE'] = 10

In [13]:
flaged_per_master_id = auto_high_df['PERSON_MASTER_ID'].to_list()
flaged_per_master_id

[]

In [14]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [15]:
summary_stg_df = pd.read_sql_query("SELECT * FROM FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG WHERE PERSON_MASTER_ID IN (SELECT Person_Master_Id FROM FINCORE_DB.Fincore_analytics.risk_person_incremental)", conn)

In [17]:
temp = pd.merge(auto_high_df,summary_stg_df , on = 'PERSON_MASTER_ID',how = 'inner')

In [19]:
temp

Empty DataFrame
Columns: [RULE_NO, RISK_RULE_LIST, RISK_SCORE, PERSON_MASTER_ID, PEER_GROUP_ID, PAN, DIN, CKYC_NUMBER, FULL_NAME, FATHER_NAME, MOTHER_NAME, DATE_OF_BIRTH, GENDER, CBDT_INCOME_RANGE, TRAN_AMNT_CREDIT, TRAN_AMNT_DEBIT, PEER_GROUP_AMNT, PINCODE, LATITUDE, LONGITUDE, OCCUPATION_BUSINESS, ACCOUNT_COUNT, ADDRESS_COUNT, MOBILE_COUNT, TELEPHONE_COUNT, EMAIL_COUNT, WATCHLIST_COUNT, LEA_REQUESTS_COUNT, LNK_REPORT_COUNT, STR_REPORT_COUNT, STR_AMOUNT_DEBIT, STR_AMOUNT_CREDIT, CTR_REPORT_COUNT, CTR_AMOUNT_DEBIT, CTR_AMOUNT_CREDIT, NTR_REPORT_COUNT, NTR_AMOUNT_DEBIT, NTR_AMOUNT_CREDIT, PTR_REPORT_COUNT, PTR_AMOUNT_DEBIT, PTR_AMOUNT_CREDIT, CBWTR_REPORT_COUNT, CBWTR_AMOUNT_DEBIT, CBWTR_AMOUNT_CREDIT, CCR_REPORT_COUNT, CCR_AMOUNT_DEBIT, CCR_AMOUNT_CREDIT, CREATE_DATE, UPDATE_DATE]
Index: []

In [20]:
# Adding the flaged person in auto high rules to risk person table

for ind,row in temp.iterrows():
    try:
        conn.execute('''INSERT INTO [Fincore_Analytics].[RISK_PERSON] (PERSON_MASTER_ID, PEER_GROUP_ID, RISK_SCORE)
    values (?,?,?)''',(row['PERSON_MASTER_ID'],row['PEER_GROUP_ID'],10))
        
    except:
        conn.execute('''update [Fincore_Analytics].[RISK_PERSON]  set PEER_GROUP_ID = '{}' , RISK_SCORE = 10 , CREATE_DATE = GETDATE() WHERE PERSON_MASTER_ID = {}  '''.format(row['PEER_GROUP_ID'],row['PERSON_MASTER_ID']))

conn.commit()
conn.close()


In [21]:
et_risk_df_per_rules = et_risk_df_per_rules[~et_risk_df_per_rules['PERSON_MASTER_ID'].isin(auto_high_df['PERSON_MASTER_ID'].to_list())]

In [23]:
et_risk_df_per_rules.drop_duplicates(inplace=True)

In [27]:
#changing vertical format to horizontal
pivot_per_df = et_risk_df_per_rules
pivot_per_df = pivot_per_df.pivot(index='PERSON_MASTER_ID', columns='RULE_NO', values=['RULE_VALUE'])
pivot_per_df.columns = [col[1] for col in pivot_per_df.columns.values]
pivot_per_df = pivot_per_df.reset_index()
cols = pivot_per_df.columns.to_list()
cols.append('CREATE_DATE')
pivot_per_df = pivot_per_df.merge(et_risk_df_per_rules,on = 'PERSON_MASTER_ID')[cols]


In [29]:
data_ind_U = data_ind_U.merge(pivot_per_df,how = 'left',on = 'PERSON_MASTER_ID')

PERSON_MASTER_ID OCCUPATION_BUSINESS     PEER_GROUP_AMNT PEER_GROUP_ID  \
0               180              Others 3674487313471.00000      Others_0   
1               180              Others 3674487313471.00000      Others_0   
2               180              Others 3674487313471.00000      Others_0   
3               180              Others 3674487313471.00000      Others_0   
4               180              Others 3674487313471.00000      Others_0   

   PEER_GROUP_AMNT_Scaled P002001001 P002001002 P002001007 P002001008  \
0                 1.00000        NaN          1          1          1   
1                 1.00000        NaN          1          1          1   
2                 1.00000        NaN          1          1          1   
3                 1.00000        NaN          1          1          1   
4                 1.00000        NaN          1          1          1   

  P002001009 P002001012 P002001018 P002001024 P002001025 P002001027  \
0        NaN          1          1          1          1          1   
1        NaN          1          1          1          1          1   
2        NaN          1          1          1          1          1   
3        NaN          1          1          1          1          1   
4        NaN          1          1          1          1          1   

  P002001030 P002001033 CREATE_DATE  
0          1          1  2023-02-23  
1          1          1  2023-02-23  
2          1          1  2023-02-23  
3          1          1  2023-02-23  
4          1          1  2023-02-23

In [30]:
data_ind_U = data_ind_U[~data_ind_U['PERSON_MASTER_ID'].isin(auto_high_df['PERSON_MASTER_ID'].to_list())]

In [31]:
data_ind_U.drop(columns = ['PERSON_MASTER_ID','CREATE_DATE'],inplace = True)

KeyError: "['PERSON_MASTER_ID.1'] not found in axis"

In [ ]:
data_ind_U.drop_duplicates(inplace = True)

In [ ]:
# reqd_cols_U = [col for col in data_ind_U.columns if 'nbr' in col.lower()]
reqd_cols_per = list(set(non_stat_rules) - set(auto_high_rules)) #doing this to remove auto high rule from nan_stat_rule_list

In [ ]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [ ]:
weight_sql_inc = pd.read_sql_query("SELECT * FROM [Fincore_Analytics].[RISK_LKP_RULES]", conn)
weight_sql_inc

In [ ]:
data_ind_U.to_csv(r"/data/Synthetic_data_Clustered_individual_U_inc.csv",index=False)

In [ ]:
print("Threshold_Computation_V2_Incremental_data Completed")